In [1]:
%matplotlib inline
import sys
sys.path
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from tqdm import tqdm_notebook
import json
from collections import Counter
from datetime import date
from multiprocessing import Process, Manager, Pool
import shutil

import xlrd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from torchvision import datasets, transforms
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
from torchvision.utils import make_grid
from torch.autograd import Variable


from ECG_Dataset_class import ECG_Dataset
from ECG_preprocessing import train_validation_test_split
from ECG_Model import ECG_Net, train_model

In [2]:
params = json.load(open('config.json', 'r'))

train_data = ECG_Dataset(txt = params["multilabel_save_folder"]+'train.txt', transform=transforms.ToTensor())
train_loader = DataLoader(dataset=train_data, batch_size=params["batch_size"], shuffle=True,num_workers=16,pin_memory=True)
val_data = ECG_Dataset(txt= params["multilabel_save_folder"]+'validation.txt', transform=transforms.ToTensor())
val_loader = DataLoader(dataset=val_data, batch_size=params["batch_size"], shuffle=False,num_workers=16,pin_memory=True)
test_data = ECG_Dataset(txt= params["multilabel_save_folder"]+'test.txt', transform=transforms.ToTensor())
test_loader = DataLoader(dataset=test_data, batch_size=params["batch_size"], shuffle=False,num_workers=16,pin_memory=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model  = ECG_Net().to(device)

tmp = {i: len(os.listdir(os.path.join(params["ecg_root_path"], j))) for i, j in enumerate(sorted(
    os.listdir(params["ecg_root_path"]), key=lambda x: int(x[0]) if x[1] == '-' else int(x[:2])))}
counter = Counter(tmp)
max_val = float(max(counter.values()))       
class_weight_tmp = {class_id : max_val/num_ecg for class_id, num_ecg in counter.items()}

c_weight = []
for key,val in class_weight_tmp.items():
    c_weight.append(val)
class_weight = torch.FloatTensor(c_weight).to(device)
Loss_fn = nn.BCELoss(weight = class_weight).to(device)

lr =  1e-4
momentum = 0.9
optimizer = torch.optim.Adam(model.parameters(),lr = lr, betas = (0.9, 0.999),eps = 1e-8, weight_decay = 1e-6)
num_epochs = 60

train_model(model,train_loader,val_loader,params["batch_size"], num_epochs,Loss_fn,optimizer)